# Transformer Benchmark

In this notebook, we will train and evaluate different transformer models on our cleaned cyberbullying tweet dataset.
We will try BERT-base, DistilBERT, and RoBERTa. We want to compare their performance in classifying tweets .


In [1]:
#imports
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import matplotlib.pyplot as plt
import seaborn as sns

device = 'cuda' if torch.cuda.is_available() else 'cpu'

df = pd.read_csv("clean_tweets.csv")

## Preprocessing

In [2]:
df = df.dropna(subset=['tweet_text'])
le = LabelEncoder()
df['label'] = le.fit_transform(df['cyberbullying_type'])

## Splitting data into training and testing sets

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['tweet_text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

## Create Dataset Class

We create a PyTorch dataset class to use with the HuggingFace Trainer

In [5]:
class CyberbullyingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item


## BERT-base

1. Tokenization

In [16]:
tokenizer_bert = AutoTokenizer.from_pretrained("bert-base-uncased")

train_encodings_bert = tokenizer_bert(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings_bert = tokenizer_bert(list(test_texts), truncation=True, padding=True, max_length=128)

train_dataset_bert = CyberbullyingDataset(train_encodings_bert, list(train_labels))
test_dataset_bert = CyberbullyingDataset(test_encodings_bert, list(test_labels))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2. Model Setup

In [17]:
model_bert = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(le.classes_)
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3. Training

In [18]:
training_args_bert = TrainingArguments(
    output_dir='./bert_results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    logging_dir='./bert_logs',
    report_to=[]
)

trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=train_dataset_bert,
    eval_dataset=test_dataset_bert,
    tokenizer=tokenizer_bert
)

trainer_bert.train()

/tmp/ipython-input-732820946.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_bert = Trainer(


Epoch,Training Loss,Validation Loss
1,0.359600,0.370921
2,0.252900,0.360307


TrainOutput(global_step=4456, training_loss=0.3464805681786803, metrics={'train_runtime': 1622.3171, 'train_samples_per_second': 43.941, 'train_steps_per_second': 2.747, 'total_flos': 3736707905000592.0, 'train_loss': 0.3464805681786803, 'epoch': 2.0})

- Train loss fell to 0.25 by end but validation rose slightly which hints at minor overfitting

4. Evaluation

In [19]:
preds_bert = trainer_bert.predict(test_dataset_bert)
y_pred_bert = np.argmax(preds_bert.predictions, axis=1)

print(classification_report(test_labels, y_pred_bert, target_names=le.classes_))

                     precision    recall  f1-score   support

                age       0.99      0.98      0.98      1588
          ethnicity       0.98      0.99      0.98      1501
             gender       0.90      0.89      0.89      1527
  not_cyberbullying       0.67      0.55      0.60      1322
other_cyberbullying       0.66      0.77      0.71      1379
           religion       0.95      0.97      0.96      1594

           accuracy                           0.87      8911
          macro avg       0.86      0.86      0.85      8911
       weighted avg       0.87      0.87      0.86      8911



- Outstanding on type specific bullying( F1 0.96–0.99) with good precision and recall

- average handling of non/vague bullying (F1 0.60–0.71) though recall for vague tweets (0.54) is moderate

## DistilBERT

1. Tokenization

In [12]:
tokenizer_distil = AutoTokenizer.from_pretrained("distilbert-base-uncased")

train_encodings_distil = tokenizer_distil(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings_distil = tokenizer_distil(list(test_texts), truncation=True, padding=True, max_length=128)

train_dataset_distil = CyberbullyingDataset(train_encodings_distil, list(train_labels))
test_dataset_distil = CyberbullyingDataset(test_encodings_distil, list(test_labels))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2. Model Setup

In [13]:
model_distil = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(le.classes_)
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3. Training

In [14]:
training_args_distil = TrainingArguments(
    output_dir='./distilbert_results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    logging_dir='./distilbert_logs',
    report_to=[]
)

trainer_distil = Trainer(
    model=model_distil,
    args=training_args_distil,
    train_dataset=train_dataset_distil,
    eval_dataset=test_dataset_distil,
    tokenizer=tokenizer_distil
)

trainer_distil.train()

/tmp/ipython-input-2605276030.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_distil = Trainer(


Epoch,Training Loss,Validation Loss
1,0.363800,0.355017
2,0.258600,0.356357


TrainOutput(global_step=4456, training_loss=0.3467179252172501, metrics={'train_runtime': 810.2524, 'train_samples_per_second': 87.98, 'train_steps_per_second': 5.5, 'total_flos': 1881370993665168.0, 'train_loss': 0.3467179252172501, 'epoch': 2.0})

4. Evaluation

In [15]:
preds_distil = trainer_distil.predict(test_dataset_distil)
y_pred_distil = np.argmax(preds_distil.predictions, axis=1)

print(classification_report(test_labels, y_pred_distil, target_names=le.classes_))

                     precision    recall  f1-score   support

                age       0.99      0.98      0.98      1588
          ethnicity       0.98      0.99      0.99      1501
             gender       0.88      0.89      0.89      1527
  not_cyberbullying       0.67      0.53      0.59      1322
other_cyberbullying       0.66      0.77      0.71      1379
           religion       0.96      0.97      0.96      1594

           accuracy                           0.87      8911
          macro avg       0.86      0.85      0.85      8911
       weighted avg       0.86      0.87      0.86      8911



- near perfect metrics on type specific bullying (F1 0.96–0.99)
- solid performance (85% accuracy)

## RoBERTa

1. Tokenization

In [7]:
tokenizer_roberta = AutoTokenizer.from_pretrained("roberta-base")

train_encodings_roberta = tokenizer_roberta(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings_roberta = tokenizer_roberta(list(test_texts), truncation=True, padding=True, max_length=128)

train_dataset_roberta = CyberbullyingDataset(train_encodings_roberta, list(train_labels))
test_dataset_roberta = CyberbullyingDataset(test_encodings_roberta, list(test_labels))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

2. Model Setup

In [8]:
model_roberta = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=len(le.classes_)
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3. Training

In [9]:
training_args_roberta = TrainingArguments(
    output_dir='./roberta_results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    logging_dir='./roberta_logs',
    report_to=[]
)

trainer_roberta = Trainer(
    model=model_roberta,
    args=training_args_roberta,
    train_dataset=train_dataset_roberta,
    eval_dataset=test_dataset_roberta,
    tokenizer=tokenizer_roberta
)

trainer_roberta.train()

/tmp/ipython-input-723298419.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_roberta = Trainer(


Epoch,Training Loss,Validation Loss
1,0.409200,0.403278
2,0.308600,0.354933


TrainOutput(global_step=4456, training_loss=0.39718807364280606, metrics={'train_runtime': 1444.2149, 'train_samples_per_second': 49.36, 'train_steps_per_second': 3.085, 'total_flos': 3480267166422120.0, 'train_loss': 0.39718807364280606, 'epoch': 2.0})

4. Evaluation

In [10]:
preds_roberta = trainer_roberta.predict(test_dataset_roberta)
y_pred_roberta = np.argmax(preds_roberta.predictions, axis=1)

print(classification_report(test_labels, y_pred_roberta, target_names=le.classes_))

                     precision    recall  f1-score   support

                age       0.98      0.98      0.98      1588
          ethnicity       0.99      0.99      0.99      1501
             gender       0.90      0.88      0.89      1527
  not_cyberbullying       0.66      0.54      0.60      1322
other_cyberbullying       0.66      0.77      0.71      1379
           religion       0.95      0.97      0.96      1594

           accuracy                           0.87      8911
          macro avg       0.86      0.86      0.85      8911
       weighted avg       0.87      0.87      0.86      8911



- near perfect on typr specific bullying (F1 0.96–0.99) with excellent balance
- solid on not bullying and vague tweets (F1 0.60–0.71)
- not cyberbullying (0.54) remains a weak point

## Saving Results

In [20]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# collect metrics for each transformer model
results = []

def add_results(name, y_true, y_pred):
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-macro": f1_score(y_true, y_pred, average='macro'),
        "F1-weighted": f1_score(y_true, y_pred, average='weighted'),
        "Precision-macro": precision_score(y_true, y_pred, average='macro'),
        "Precision-weighted": precision_score(y_true, y_pred, average='weighted'),
        "Recall-macro": recall_score(y_true, y_pred, average='macro'),
        "Recall-weighted": recall_score(y_true, y_pred, average='weighted')
    })

# add transformer models
add_results("BERT-base", test_labels, y_pred_bert)
add_results("DistilBERT", test_labels, y_pred_distil)
add_results("RoBERTa", test_labels, y_pred_roberta)

# create dataframe
results_df = pd.DataFrame(results)

# display resylts
print(results_df)

# save to CSV
results_df.to_csv("transformer_metrics.csv", index=False)

        Model  Accuracy  F1-macro  F1-weighted  Precision-macro  \
0   BERT-base  0.866008  0.854805     0.864337         0.857142   
1  DistilBERT  0.865111  0.853237     0.862966         0.855739   
2     RoBERTa  0.865784  0.854513     0.864104         0.856377   

   Precision-weighted  Recall-macro  Recall-weighted  
0            0.866012      0.856154         0.866008  
1            0.864632      0.854943         0.865111  
2            0.865348      0.855862         0.865784  


- BERT-base and RoBERTa edge out DistilBERT in most metrics showing robust contextual understanding for attribute specific bullying